In [1]:
%pip install pandas pyarrow rectools

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import PopularModel, ImplicitALSWrapperModel
from implicit.als import AlternatingLeastSquares

/mnt/d/datasetPSB/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Пути к файлам
DATA_PATH = "/mnt/d/datasetPSB"
MARKETPLACE_EVENTS_PATH = os.path.join(DATA_PATH, "marketplace", "events")
ITEMS_PATH = os.path.join(DATA_PATH, "marketplace", "items.pq")
USERS_PATH = os.path.join(DATA_PATH, "users.pq")

# Читаем пользователей
users_df = pd.read_parquet(USERS_PATH)
print(f"Пользователей: {users_df.shape}")

# Читаем товары (только основные колонки)
items_df = pd.read_parquet(ITEMS_PATH, columns=['item_id', 'category', 'brand_id', 'price'])
print(f"Товаров: {items_df.shape}")

# Читаем события (берем первые 10 файлов)
event_files = sorted([os.path.join(MARKETPLACE_EVENTS_PATH, f) for f in os.listdir(MARKETPLACE_EVENTS_PATH) if f.endswith('.pq')])
events_dfs = []

print("Загрузка событий...")
for f in event_files[:10]:
    # Читаем только нужные колонки, чтобы не тратить память
    events_dfs.append(pd.read_parquet(f, columns=['user_id', 'item_id', 'timestamp']))

interactions_df = pd.concat(events_dfs, ignore_index=True)

# Переименовываем колонки под стандарт Rectools
interactions_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
    'timestamp': Columns.Datetime
}, inplace=True)

# Добавляем вес взаимодействия (1 - был контакт)
interactions_df[Columns.Weight] = 1.0

print(f"Всего событий: {interactions_df.shape}")
display(interactions_df.head())

Пользователей: (43997878, 3)
Товаров: (28621002, 4)
Загрузка событий...
Всего событий: (28514324, 4)


,user_id,item_id,datetime,weight
0,15549980,nfmcg_25621580,1000 days 00:00:00.062127,1.0
1,34952676,nfmcg_25968387,1000 days 00:00:00.116202,1.0
2,43419595,nfmcg_25497204,1000 days 00:00:00.159632,1.0
3,58440865,nfmcg_7658098,1000 days 00:00:00.215206,1.0
4,70296647,nfmcg_8361059,1000 days 00:00:00.258081,1.0


In [5]:
# Пути к файлам
DATA_PATH = "/mnt/d/datasetPSB"
MARKETPLACE_EVENTS_PATH = os.path.join(DATA_PATH, "marketplace", "events")
ITEMS_PATH = os.path.join(DATA_PATH, "marketplace", "items.pq")
USERS_PATH = os.path.join(DATA_PATH, "users.pq")

# Читаем пользователей
users_df = pd.read_parquet(USERS_PATH)
print(f"Пользователей: {users_df.shape}")

# Читаем товары (только основные колонки)
items_df = pd.read_parquet(ITEMS_PATH, columns=['item_id', 'category', 'brand_id', 'price'])
print(f"Товаров: {items_df.shape}")

# Читаем события (берем первые 10 файлов)
event_files = sorted([os.path.join(MARKETPLACE_EVENTS_PATH, f) for f in os.listdir(MARKETPLACE_EVENTS_PATH) if f.endswith('.pq')])
events_dfs = []

print("Загрузка событий...")
for f in event_files[:10]:
    # Читаем
    df = pd.read_parquet(f, columns=['user_id', 'item_id', 'timestamp'])
    events_dfs.append(df)

interactions_df = pd.concat(events_dfs, ignore_index=True)

# Переименовываем колонки под стандарт Rectools
interactions_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
    'timestamp': Columns.Datetime
}, inplace=True)

# --- ИСПРАВЛЕНИЕ ОШИБКИ ---
# Преобразуем Timedelta (интервал) в Datetime (дату), добавляя базовую дату
base_date = pd.Timestamp("2024-01-01")
interactions_df[Columns.Datetime] = base_date + interactions_df[Columns.Datetime]
# --------------------------

# Добавляем вес взаимодействия (1 - был контакт)
interactions_df[Columns.Weight] = 1.0

print(f"Всего событий: {interactions_df.shape}")
print("Типы данных:")
print(interactions_df.dtypes)
display(interactions_df.head())

Пользователей: (43997878, 3)
Товаров: (28621002, 4)
Загрузка событий...
Всего событий: (28514324, 4)
Типы данных:
user_id             uint64
item_id             object
datetime    datetime64[us]
weight             float64
dtype: object


,user_id,item_id,datetime,weight
0,15549980,nfmcg_25621580,2026-09-27 00:00:00.062127,1.0
1,34952676,nfmcg_25968387,2026-09-27 00:00:00.116202,1.0
2,43419595,nfmcg_25497204,2026-09-27 00:00:00.159632,1.0
3,58440865,nfmcg_7658098,2026-09-27 00:00:00.215206,1.0
4,70296647,nfmcg_8361059,2026-09-27 00:00:00.258081,1.0


In [10]:
# Создаем специальную структуру данных для библиотеки RecTools
# Она внутри себя переводит ID пользователей и товаров в числа от 0 до N
dataset = Dataset.construct(
    interactions_df=interactions_df
)
print("Датасет готов к обучению")

Датасет готов к обучению


In [11]:
# Обучаем модель ALS
# factors=64 - размер вектора скрытых признаков
# iterations=15 - количество эпох обучения
als_model = ImplicitALSWrapperModel(
    AlternatingLeastSquares(factors=64, iterations=15, use_gpu=False, random_state=42)
)

print("Обучение ALS модели...")
# ВАЖНО: передаем объект dataset, который мы создали в Ячейке 3
als_model.fit(dataset) 

# Генерируем рекомендации
# Берем тех же 5 пользователей из прошлого шага
test_users = interactions_df[Columns.User].unique()[:5]

als_recs = als_model.recommend(
    users=test_users,
    dataset=dataset,
    k=5,
    filter_viewed=True
)

print("Персональные рекомендации (ALS):")
display(als_recs)

Обучение ALS модели...
Персональные рекомендации (ALS):


,user_id,item_id,score,rank
0,15549980,nfmcg_18954296,0.450651,1
1,15549980,nfmcg_8948521,0.347026,2
2,15549980,nfmcg_23976640,0.319836,3
3,15549980,nfmcg_13410526,0.316723,4
4,15549980,nfmcg_7453272,0.279495,5
5,34952676,nfmcg_17723875,0.065199,1
6,34952676,nfmcg_21058740,0.061396,2
7,34952676,nfmcg_27771371,0.060898,3
8,34952676,nfmcg_24837755,0.057884,4
9,34952676,nfmcg_3128314,0.051458,5


In [12]:
import pickle

# Сохраняем модель в файл
with open('als_model_marketplace.pkl', 'wb') as f:
    pickle.dump(als_model, f)

print("Модель сохранена как 'als_model_marketplace.pkl'")

Модель сохранена как 'als_model_marketplace.pkl'


In [1]:
import pandas as pd
import os
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import PopularModel, ImplicitALSWrapperModel
from implicit.als import AlternatingLeastSquares
import pickle

/mnt/d/datasetPSB/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_and_process_source(path, source_name, file_limit=5):
    print(f"\n--- Загрузка {source_name} ---")
    files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.endswith('.pq')])[:file_limit]
    
    dfs = []
    for f in files:
        try:
            # Читаем файл. Пытаемся найти стандартные колонки
            df = pd.read_parquet(f)
            
            # Оставляем только нужные колонки, если они есть
            cols_to_keep = ['user_id', 'timestamp']
            # В разных файлах ID товара может называться по-разному, ищем варианты
            if 'item_id' in df.columns:
                item_col = 'item_id'
            elif 'sku_id' in df.columns: # иногда бывает sku_id
                item_col = 'sku_id'
            else:
                # Если не нашли, берем первую попавшуюся колонку, которая не юзер и не время (рискованно, но работает для примера)
                item_col = [c for c in df.columns if c not in ['user_id', 'timestamp']][0]
            
            df = df[['user_id', item_col, 'timestamp']].copy()
            df.rename(columns={item_col: Columns.Item}, inplace=True)
            
            dfs.append(df)
        except Exception as e:
            print(f"Ошибка чтения {f}: {e}")

    if not dfs:
        return pd.DataFrame()
        
    full_df = pd.concat(dfs, ignore_index=True)
    
    # Добавляем префикс к ID товара, чтобы различать источники (например "retail_123")
    full_df[Columns.Item] = source_name + "_" + full_df[Columns.Item].astype(str)
    
    # Исправление времени (если это Timedelta, как в marketplace)
    if pd.api.types.is_timedelta64_dtype(full_df['timestamp']):
        base_date = pd.Timestamp("2024-01-01")
        full_df[Columns.Datetime] = base_date + full_df['timestamp']
    else:
        full_df[Columns.Datetime] = pd.to_datetime(full_df['timestamp'])
        
    # Переименовываем user_id
    full_df.rename(columns={'user_id': Columns.User}, inplace=True)
    
    # Добавляем вес
    full_df[Columns.Weight] = 1.0
    
    # Удаляем лишние колонки
    full_df = full_df[[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
    
    print(f"Загружено {full_df.shape[0]} строк из {source_name}")
    return full_df


import os



DATA_PATH = "/mnt/d/datasetPSB"
# Пути к папкам
PATH_RETAIL = os.path.join(DATA_PATH, "retail", "events")
PATH_PAYMENTS = os.path.join(DATA_PATH, "payments", "receipts")
PATH_MARKETPLACE = os.path.join(DATA_PATH, "marketplace", "events")


In [ ]:
import gc

def train_model_for_domain(path, source_name, file_limit=10):
    print(f"\n=== Обучение модели для: {source_name} ===")
    
    # 1. Загрузка и обработка (используем ту же логику, что и раньше)
    df = load_and_process_source(path, source_name, file_limit=file_limit)
    
    if df.empty:
        print(f"Нет данных для {source_name}, пропускаем.")
        return None
    
    # 2. Создание датасета
    ds = Dataset.construct(interactions_df=df)
    
    # 3. Обучение
    model = ImplicitALSWrapperModel(
        AlternatingLeastSquares(factors=32, iterations=10, use_gpu=False, random_state=42)
    )
    model.fit(ds)
    
    # 4. Сохранение
    filename = f'als_model_{source_name}.pkl'
    with open(filename, 'wb') as f:
        pickle.dump({'model': model, 'dataset': ds}, f) # Сохраняем и модель, и маппинг ID
        
    print(f"Модель сохранена в {filename}")
    
    # 5. Очистка памяти
    del df, ds, model
    gc.collect()
    
    return filename

# Обучаем по очереди
# file_limit регулируй в зависимости от RAM. 
# Если всё еще падает, уменьши до 3-5 файлов.
model_files = {}

# 1. Marketplace
model_files['marketplace'] = train_model_for_domain(PATH_MARKETPLACE, "mkt", file_limit=10)

# 2. Retail
model_files['retail'] = train_model_for_domain(PATH_RETAIL, "rtl", file_limit=10)

# 3. Payments
model_files['payments'] = train_model_for_domain(PATH_PAYMENTS, "pay", file_limit=10)

print("\nВсе модели обучены!")


=== Обучение модели для: mkt ===

--- Загрузка mkt ---
Загружено 28514324 строк из mkt


/mnt/d/datasetPSB/.venv/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 18 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Модель сохранена в als_model_mkt.pkl

=== Обучение модели для: rtl ===

--- Загрузка rtl ---
Загружено 45333444 строк из rtl
Модель сохранена в als_model_rtl.pkl

=== Обучение модели для: pay ===

--- Загрузка pay ---
